In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In this document we will be discussing approaches to investigate neuronal correlations and synchrony using correlograms.

We will first focus on a more theoretical paper: 

### Perkel et al (1967) *Neuronal Spike Trains and Stochastic Point Processes I*

In the present work, we deal with spike trains which are viewed as indistinguishable, "all-or-none" events occurring over time and exhibiting a high degree of variability or seeming randomness. Thus, we treat these spike trains as *stochastic point processes*.

In point processes where events are indistinguishable (a spike being present) except the time of occurrence of the event, the intervals between events take on the properties of random variables, and as such are considered to be drawn (not necessarily independently) from a certain probability distribution. First, we deal with *stationary* point processes, where the parameters of the distribution generating the interspike intervals are steady over time.

A *renewal process* is a stationary point process in which the intervals between events are independent.

**Order-Idenependent Statistical Measures**

For renewal point processes, the (marginal) distribution of intervals between successive events completely characterizes the process. The interspike-interval histogram

###Brody (1999) *Correlations without Synchrony*

A standard method for determining the presence of interactions between the spike trains of two neurons is to calculate a *shuffle-corrected cross-correlogram*, or *co-variogram*. Stricly speaking, peaks in covariograms reflect that the spike trains are not independent, but this dependence could arise from multiple sources. This paper focuses on three such sources, *spike-timing*, *latency*, and *excitability* covariations.

$S_i^r(t)$ is the spike train of cell $i$, on trial $r$, where the spike train is a function of time. The *cross-correlogram* for each trial is given by:

$$
C^r(\tau) = \sum_{t = -\infty}^{\infty}S_1^r(t)S_2^r(t+\tau) = S_1^r \odot S_2^r
$$

We now define $P_i(t) = \langle S_i^r(t)\rangle$, where $\langle \rangle$ denotes an average across trials, $r$. This is the PSTH of neuron $i$ if the times are relative to a stimulus. We then define the covariogram of $S_1$ and $S_2$ as:

$$
V = \langle (S_1^r - P_1) \odot (S_2^r - P_2) \rangle = \langle S_1^r \odot S_2^r \rangle - P_1 \odot P_2 = R - K,
$$

where $R$ is the *raw cross-correlogram* and $K$ is the *shuffle-corrector*. This can be verified by expanding the original expression and pushing through the sums, such that, for example:

$$
\langle \sum_t P_1(t)S_2^r(t+\tau) \rangle = \sum_t P_1(t) \langle S_2^r(t+\tau) \rangle = \sum_t P_1(t) P_2(t+\tau) = P_1 \odot P_2.
$$

If $S_1$ and $S_2$ are independent, then $E[V] = 0$, so departures from $0$ indicate dependence. One may ask what counts as a significant departure from $0$, however, and answering this requires further assumptions. We would additionally assume that different trials $r$ are independent of one another and different bins within a trial are independent of one another. Under these circumstances one can calculate the variance of $V$ 

$$
\sigma_V^2 = \frac{1}{N_{trials}}(\sigma_1^2 \odot \sigma_2^2 + P_1^2 \odot \sigma_2^2 + P_2^2 \odot \sigma_1^2)
$$

where sample means and variances are most typically used. A $2\sigma$ limit is reasonable, particularly is a further Gaussian assumption is made, as this would then correspond to 95% CI's. To verify the above expression, consider the variance of a product of two independent scalars $x$ and $y$:

$$
Var(xy) = E[x^2]E[y^2] - E[x]^2E[y]^2 = (\sigma_x^2 + p_x^2)(\sigma_y^2 + p_y^2) - p_x^2p_y^2 = \sigma_x^2\sigma_y^2 + \sigma_x^2p_y^2 + \sigma_y^2p_x^2
$$

We will also make use of the Joint Peri-Stimulus Time Histogram (JPSTH), giving a matrix of covariances

$$
J(t_1,t_2) = \langle S_1^r(t_1)S_2^r(t_2) \rangle - \langle S_1^r(t_1) \rangle \langle S_2^r(t_2) \rangle,
$$

which we can normalize to give a matrix of correlation coefficients

$$
J_N(t_1,t_2) = \frac{J(t_1,t_2)}{\sigma_1(t_1)\sigma_2(t_2)}
$$

**Latency Covariations**

We will first examine how latency covariations impact the covariogram. We take two neurons and simulate responses to a stimulus over 200 trials from a Poisson distribution

In [31]:
nT = 200
t = arange(1000)
rate = 40
u1 = random.uniform(0,1,(nT,1000))
u2 = random.uniform(0,1,(nT,1000))
sp1 = zeros(u1.shape)
sp2 = sp1
sp1[(rate*.001) > u1] = 1
sp2[(rate*.001) > u2] = 1
sum(sp1[0,:])

84.0

### CCG Correction Methods

Calculate CCG as:

$$
\frac{\frac{1}{M}\Sigma_{i=1}^M \Sigma_{t=1}^N x_1^i(t)x_2^i(t + \tau)}{\theta(\tau)\sqrt{\lambda_1\lambda_2}}
$$

**Shift Predictor Correction (Kohn and Smith, 2005)**

Calculated by: subtracting the CCG calculated by using spike trains that are offset by one trial

Corrects for: correlations induced by the stimulus

**Jitter correction (Smith and Kohn, 2008)** 

Calculated by: subtracting expected value of CCGs produced from resampling the original data with spike times randomly perturbed (jittered) within a small temporal window; then average over all possible resamples

Corrects for: temporal correlations on timescales larger than the jitter window and stimulus-locked correlations

Jitter correction preserves firing rates within the jitter window bin and preserves the PSTH

Analysis in Smith and Kohn (2008) revealed two components to CCG, one which is a sharp peak and one that is more broad. Both peaks decay with distance between neurons in the pair, however, the sharp peak decays more rapidly. 

The authors used jitter-correction to examine CCG peaks, specifically. They use jitter windows of 50ms. This peak is likely due to direct connectivity between neurons in the pair given the short timescale and the decay with distance (as connection probability also decays with distance).

